In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("train_ver2.csv", nrows=7000000, low_memory=False)

In [3]:
print(f"train shape: {df.shape}")

train shape: (7000000, 48)


In [4]:
df.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta             

In [6]:
hd = df.head(10)

In [7]:
hd.iloc[:, :24]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO
5,2015-01-28,1050615,N,ES,H,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,45.0,TOLEDO,0.0,22220.04,03 - UNIVERSITARIO
6,2015-01-28,1050616,N,ES,H,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,24.0,LEON,0.0,295590.36,03 - UNIVERSITARIO
7,2015-01-28,1050617,N,ES,H,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,113316.66,03 - UNIVERSITARIO
8,2015-01-28,1050619,N,ES,H,24,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,20.0,GIPUZKOA,0.0,NaN,03 - UNIVERSITARIO
9,2015-01-28,1050620,N,ES,H,23,2012-08-10,0.0,35,1.0,...,N,NaN,KHE,N,1.0,10.0,CACERES,0.0,113194.98,03 - UNIVERSITARIO


In [8]:
hd.iloc[:, 24:]

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.0,0.0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [9]:
df.isna().sum().sort_values(ascending=False).head(25)

conyuemp                 6999002
ult_fec_cli_1t           6989049
renta                    1252256
segmento                  120324
canal_entrada             119252
indrel_1mes               104804
tiprel_1mes               104804
nomprov                    64090
cod_prov                   64090
sexo                       27768
indfall                    27734
indext                     27734
ind_actividad_cliente      27734
tipodom                    27734
indresi                    27734
indrel                     27734
ind_nuevo                  27734
fecha_alta                 27734
pais_residencia            27734
ind_empleado               27734
ind_nomina_ult1            16063
ind_nom_pens_ult1          16063
ind_pres_fin_ult1              0
ind_ecue_fin_ult1              0
ind_fond_fin_ult1              0
dtype: int64

In [10]:
df = df.drop(columns=["conyuemp", "ult_fec_cli_1t"], inplace=False)

In [11]:
df["fecha_dato_copy"] = df["fecha_dato"]

In [12]:
def date_handler(df):
    if "fecha_alta" in df.columns:
        df.dropna(subset=["fecha_alta"], inplace=True)
        df["fecha_alta"] = pd.to_datetime(
            df["fecha_alta"], format="%Y-%m-%d", errors="coerce"
        )
    if "fecha_dato" in df.columns:
        df["fecha_dato"] = pd.to_datetime(
            df["fecha_dato"], format="%Y-%m-%d", errors="coerce"
        )

In [13]:
def filter_no_change(df):
    df.sort_values(["ncodpers", "fecha_dato"], inplace=True)
    
    product_cols = [
        'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
        'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
        'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
        'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
        'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'
    ]
    
    df["product_sum"] = df[product_cols].sum(axis=1)
    df["prev_sum"] = df.groupby("ncodpers")["product_sum"].shift(1)
    df = df[df["product_sum"] != df["prev_sum"]]
    df.drop(columns=["product_sum", "prev_sum"], inplace=True)
    return df


In [14]:
target_cols = df.columns[24:48].tolist()

In [15]:
df["fecha_dato_copy"] = df["fecha_dato"]

In [16]:
def process_dates(df):
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"], errors="coerce")
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], errors="coerce")
    df["year"] = df["fecha_dato"].dt.year
    df["month"] = df["fecha_dato"].dt.month
    df["alta_year"] = df["fecha_alta"].dt.year
    df["alta_month"] = df["fecha_alta"].dt.month
    df.drop(columns=["fecha_dato", "fecha_alta"], inplace=True)
    return df


df = process_dates(df)

In [17]:
def add_lag_features(df, product):
    df.sort_values(["ncodpers", "fecha_dato_copy"], inplace=True)
    df[f"{product}_lag"] = df.groupby("ncodpers")[product].shift(1).fillna(0)
    return df

In [18]:
for col in target_cols:
    df = add_lag_features(df, col)

In [19]:
df["renta"].fillna(df["renta"].mean(), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

In [20]:
cat_cols = df.select_dtypes(include="object").columns
for col in cat_cols:
    lbl = LabelEncoder()
    df[col] = lbl.fit_transform(df[col].astype(str))

In [21]:
X = df.drop(columns=["ncodpers"] + target_cols)
y = df[target_cols]

In [22]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
kmeans = KMeans(n_clusters=3, random_state=42)
df["cluster"] = kmeans.fit_predict(X_scaled)
X["cluster"] = df["cluster"]

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
models = {}
y_pred = []

for product in target_cols:
    model = xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        n_estimators=100,
        max_depth=7,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.7,
        use_label_encoder=False,
        random_state=42,
    )
    model.fit(X_train, y_train[product])
    models[product] = model
    y_pred.append(model.predict(X_val))

y_pred = np.array(y_pred).T

In [30]:
for col in target_cols:
    unique_vals = y_val[col].unique()
    print(f"{col} unique values: {unique_vals}")



# for i, col in enumerate(target_cols):
#     acc = accuracy_score(y_val.iloc[:, i], y_pred[:, i])
#     f1 = f1_score(y_val.iloc[:, i], y_pred[:, i], average='binary', zero_division=0)
#     print(f"{col}: Accuracy={acc:.4f}, F1 Score={f1:.4f}")



ind_cco_fin_ult1 unique values: [1 0]
ind_cder_fin_ult1 unique values: [0 1]
ind_cno_fin_ult1 unique values: [0 1]
ind_ctju_fin_ult1 unique values: [0 1]
ind_ctma_fin_ult1 unique values: [0 1]
ind_ctop_fin_ult1 unique values: [0 1]
ind_ctpp_fin_ult1 unique values: [1 0]
ind_deco_fin_ult1 unique values: [0 1]
ind_deme_fin_ult1 unique values: [0 1]
ind_dela_fin_ult1 unique values: [0 1]
ind_ecue_fin_ult1 unique values: [0 1]
ind_fond_fin_ult1 unique values: [0 1]
ind_hip_fin_ult1 unique values: [1 0]
ind_plan_fin_ult1 unique values: [0 1]
ind_pres_fin_ult1 unique values: [0 1]
ind_reca_fin_ult1 unique values: [0 1]
ind_tjcr_fin_ult1 unique values: [1 0]
ind_valo_fin_ult1 unique values: [0 1]
ind_viv_fin_ult1 unique values: [0 1]
ind_nomina_ult1 unique values: [0. 1.]
ind_nom_pens_ult1 unique values: [0. 1.]
ind_recibo_ult1 unique values: [1 0]
fecha_dato_copy unique values: [1 0 8 5 3 4 7 6 9 2]


In [31]:
test_df = pd.read_csv("test_ver2.csv", low_memory=False)

In [32]:
print(f"test shape: {test_df.shape}")

test shape: (929615, 24)


In [33]:
test_df.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [34]:
test_df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [35]:
test_df.isna().sum().sort_values(ascending=False).head(10)

conyuemp          929511
ult_fec_cli_1t    927932
nomprov             3996
cod_prov            3996
segmento            2248
canal_entrada       2081
indrel_1mes           23
tiprel_1mes           23
sexo                   5
antiguedad             0
dtype: int64

In [36]:
test_df = test_df.drop(columns=["conyuemp", "ult_fec_cli_1t"], inplace=False)

In [37]:
test_df = process_dates(test_df)
test_df.fillna(df.mode().iloc[0], inplace=True)

In [41]:
cat_cols = [col for col in cat_cols if col in test_df.columns]

for col in cat_cols:
    lbl = LabelEncoder()
    test_df[col] = lbl.fit_transform(test_df[col].astype(str))


In [ ]:
missing_cols = set(X_train.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0  

X_test = test_df[X_train.columns]


In [ ]:
for col in X_test.columns:
    if X_test[col].dtype == 'object':
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')  

X_test.fillna(0, inplace=True)


In [48]:
y_test_proba = np.array(
    [models[product].predict_proba(X_test)[:, 1] for product in target_cols]
).T
top_7_products = np.argsort(-y_test_proba, axis=1)[:, :7]
added_products = [" ".join([target_cols[i] for i in row]) for row in top_7_products]


In [49]:
submission_df = pd.DataFrame(
    {"ncodpers": test_df["ncodpers"], "added_products": added_products}
)
submission_df.to_csv("submission.csv", index=False)
